Importing Modules

In [1]:
import torch
import torchvision
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import os
import time
import numpy as np
import argparse
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
matplotlib.style.use('ggplot')
import sys, importlib as impL
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery')
import helperFuncs as funcH
import pandas as pd

/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
def calc_bottleneck_acc(bottleneck_vec, lab_vec):
    pred_vec = np.argmax(bottleneck_vec.T, axis=0).T.squeeze()
    centroid_info_pdf = funcH.get_cluster_centroids(bottleneck_vec, pred_vec, kluster_centers=None, verbose=0)
    _confMat_preds, kluster2Classes, kr_pdf, weightedPurity, cnmxh_perc = funcH.countPredictionsForConfusionMat(lab_vec, pred_vec, centroid_info_pdf=centroid_info_pdf, labelNames=None)
    sampleCount = np.sum(np.sum(_confMat_preds))
    acc = 100 * np.sum(np.diag(_confMat_preds)) / sampleCount
    bmx, bmn = np.max(bottleneck_vec), np.min(bottleneck_vec)
    return acc, bmx, bmn

funcH.setPandasDisplayOpts()

Constructing the Argument Parsers

In [3]:
#ap = argparse.ArgumentParser()
#ap.add_argument('-e', '--epochs', type=int, default=10, help='number of epochs to train our network for')
#ap.add_argument('-l', '--reg_param', type=float, default=0.001, help='regularization parameter `lambda`')
#ap.add_argument('-sc', '--add_sparse', type=str, default='yes', help='whether to add sparsity contraint or not')
#args = vars(ap.parse_args())
epochs = 100  # args['epochs']
reg_param = 0.001  # args['reg_param']
add_sparsity = 'yes'  # args['add_sparse']
learning_rate = 1e-4
batch_size = 32
print(f"Add sparsity regularization: {add_sparsity}")

Add sparsity regularization: yes


here I will change the data loader per my need

In [4]:
# get the computation device
def get_device():
    return 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = get_device()

In [5]:
# image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

FOLDERS = {
    "data": '/mnt/USB_HDD_1TB/Datasets',
    "experiment": '/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/experiments/SPARSE_TORCH/sparse_torch_ae_06',
}
FOLDERS["model_save"] = os.path.join(FOLDERS["experiment"], "model")
FOLDERS["decoder_image_path_tr"] = os.path.join(FOLDERS["experiment"], "output_images_tr")
FOLDERS["decoder_image_path_va"] = os.path.join(FOLDERS["experiment"], "output_images_va")
funcH.createDirIfNotExist(FOLDERS["model_save"])
funcH.createDirIfNotExist(FOLDERS["decoder_image_path_tr"])
funcH.createDirIfNotExist(FOLDERS["decoder_image_path_va"])

trainset = datasets.FashionMNIST(
    root=FOLDERS["data"],
    train=True, 
    download=True,
    transform=transform
)
testset = datasets.FashionMNIST(
    root=FOLDERS["data"],
    train=False,
    download=True,
    transform=transform
)
 
# trainloader
trainloader = DataLoader(
    trainset, 
    batch_size=batch_size,
    shuffle=True
)
#testloader
testloader = DataLoader(
    testset, 
    batch_size=batch_size, 
    shuffle=False
)

In [6]:
# define the autoencoder model
class SparseAutoencoder(nn.Module):
    def __init__(self, loss_type):
        super(SparseAutoencoder, self).__init__()
 
        # encoder
        self.enc1 = nn.Linear(in_features=784, out_features=256)
        self.enc2 = nn.Linear(in_features=256, out_features=128)
        self.enc3 = nn.Linear(in_features=128, out_features=64)
        self.enc4 = nn.Linear(in_features=64, out_features=32)
        self.enc5 = nn.Linear(in_features=32, out_features=16)
 
        # decoder 
        self.dec1 = nn.Linear(in_features=16, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=128)
        self.dec4 = nn.Linear(in_features=128, out_features=256)
        self.dec5 = nn.Linear(in_features=256, out_features=784)
        
        self.loss_type=loss_type
        self.device = get_device()
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        bottleneck = F.relu(self.enc5(x))

        # decoding
        x = F.relu(self.dec1(bottleneck))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        return x, bottleneck
model = SparseAutoencoder(loss_type='kl').to(device)

In [7]:
# the loss function
criterion = nn.MSELoss()
# the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# get the layers as a list
model_children = list(model.children())
[print(i) for i in model_children]

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=128, bias=True)
Linear(in_features=128, out_features=64, bias=True)
Linear(in_features=64, out_features=32, bias=True)
Linear(in_features=32, out_features=16, bias=True)
Linear(in_features=16, out_features=32, bias=True)
Linear(in_features=32, out_features=64, bias=True)
Linear(in_features=64, out_features=128, bias=True)
Linear(in_features=128, out_features=256, bias=True)
Linear(in_features=256, out_features=784, bias=True)


[None, None, None, None, None, None, None, None, None, None]

In [9]:
def loss_l1(bottleneck):
    return torch.mean(torch.abs(bottleneck))

def loss_l2(bottleneck):
    return torch.mean(torch.pow(bottleneck, torch.tensor(2.0).to(device))).sqrt()

def kl_divergence(bottleneck):
    rho = 0.05
    bottleneck = torch.mean(torch.sigmoid(bottleneck), 1)  # sigmoid because we need the probability distributions
    rho = torch.tensor([rho] * len(bottleneck)).to(device)
    loss_ret_1 = torch.nn.functional.kl_div(bottleneck, rho, reduction='batchmean')
    # torch.sum(rho * torch.log(rho / bottleneck) + (1 - rho) * torch.log((1 - rho) / (1 - bottleneck)))
    return loss_ret_1

In [10]:
# define the sparse loss function
def sparse_loss(autoencoder, images, print_info, loss_type):
    loss = 0
    values = images
    for i in range(len(model_children)):
        values = F.relu((model_children[i](values)))
        #if print_info:
            #print(i, ' shape=', values.shape)
        if loss_type=='l1':
            loss += loss_l1(values)
        if loss_type=='l2':
            loss += loss_l2(values)
        if loss_type=='kl':
            loss += kl_divergence(values)
        if print_info:
            print(loss_type,loss)
    return loss

In [11]:
def save_decoded_image(img, name):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, name)

# define the training function
def fit(model, dataloader, epoch, print_losses_fit):
    print('TrEpoch({:03d}) - '.format(epoch), end='')
    model.train()
    running_loss = 0.0
    
    lab_vec = []
    bottleneck_vec = []
    sparsity_loss_sum = 0
    mse_sum = 0
       
    for data in dataloader:
        img, lb = data
        lab_vec.append(lb)
        
        img = img.to(device)
        img = img.view(img.size(0), -1)
        optimizer.zero_grad()
        outputs, bottleneck = model(img)
        bottleneck_vec.append(bottleneck)
        mse_loss = criterion(outputs, img)
        mse_sum += mse_loss.item()
        #if print_losses_fit:
            #print("mse_loss:", mse_loss.to('cpu'))
            #print("bottleneck:", bottleneck.to('cpu'))
        if add_sparsity == 'yes':
            sp_loss = sparse_loss(model, img, print_losses_fit, model.loss_type)
            sparsity_loss_sum += sp_loss.item()
            # add the sparsity penalty
            if print_losses_fit:
                print("sp_loss:", sparsity_loss_sum)
            loss = mse_loss - reg_param * sp_loss
        else:
            loss = mse_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print_losses_fit = False
    
    lab_vec = np.asarray(torch.cat(lab_vec).to(torch.device('cpu')))
    bottleneck_vec = np.asarray(torch.cat(bottleneck_vec).to(torch.device('cpu')).detach().numpy())
    acc, bmx, bmn = calc_bottleneck_acc(bottleneck_vec, lab_vec)
    #print("tr bottleneck accuracy=", acc, ", max=", bmx, ", min=", bmn, ", sparsity_loss_sum=", sparsity_loss_sum)
  
    result_df = pd.DataFrame(np.array([[acc, bmx, bmn, mse_sum, sparsity_loss_sum, running_loss]]), columns=['acc','bmx','bmn','mse','spr','run'])
    #print(df.iloc[0]['mse']) #'acc','bmx','bmn','mse','spr','run'
    print("\n",result_df)
    if epoch % 2 == 0:
        difn = os.path.join(FOLDERS["decoder_image_path_tr"], "train"+str(epoch).zfill(3)+".png")
        save_decoded_image(outputs.cpu().data, difn)
    return result_df

In [12]:
# define the validation function
def validate(model, dataloader, epoch, print_losses_fit):
    print('ValEpoch({:03d}) - '.format(epoch), end='')
    model.eval()
    running_loss = 0.0
    lab_vec = []
    bottleneck_vec = []
    with torch.no_grad():
        for data in dataloader:
            img, lb = data
            lab_vec.append(lb)
            img = img.to(device)
            img = img.view(img.size(0), -1)
            outputs, bottleneck = model(img)
            bottleneck_vec.append(bottleneck)
            loss = criterion(outputs, img)
            running_loss += loss.item()
    # save the reconstructed images every 5 epochs
    lab_vec = np.asarray(torch.cat(lab_vec).to(torch.device('cpu')))
    bottleneck_vec = np.asarray(torch.cat(bottleneck_vec).to(torch.device('cpu')).detach().numpy())
    acc, bmx, bmn = calc_bottleneck_acc(bottleneck_vec, lab_vec)

    result_df = pd.DataFrame(np.array([[acc, bmx, bmn, running_loss]]), columns=['acc','bmx','bmn','run'])
    print("\n",result_df)
    
    if epoch % 2 == 0:
        outputs = outputs.view(outputs.size(0), 1, 28, 28).cpu().data
        difn = os.path.join(FOLDERS["decoder_image_path_va"], "reconstruction"+str(epoch).zfill(3)+".png")
        save_image(outputs, difn)
    return result_df

In [13]:
# train and validate the autoencoder neural network
start = time.time()
print_losses_fit = True

train_loss = []
trn_spars_loss = []
trn_bot_acc = []
val_loss = []
val_bot_acc = []

result_df_tr_all = pd.DataFrame(columns=['acc','bmx','bmn','mse','spr','run'])
result_df_va_all = pd.DataFrame(columns=['acc','bmx','bmn','run'])

print("stae6 - lr 1e-4")

for epoch in range(epochs):
    print(f"*****\n Epoch {epoch} of {epochs}")
    result_df_tr = fit(model, trainloader, epoch, print_losses_fit)
    result_df_va = validate(model, testloader, epoch, print_losses_fit)
    print_losses_fit = epoch%5==0 and epoch>0
    result_df_tr_all = result_df_tr_all.append(result_df_tr, ignore_index=True)
    result_df_va_all = result_df_va_all.append(result_df_va, ignore_index=True)
    
end = time.time()
 
print(f"{(end-start)/60:.3} minutes")
# save the trained model

mofn = os.path.join(FOLDERS["model_save"], "sparse_ae_"+str(epoch).zfill(3)+".pth")
torch.save(model.state_dict(), mofn)

stae6 - lr 1e-4
*****
 Epoch 1 of 100
TrEpoch(000) - kl tensor(-0.1759, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-0.3511, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-0.5262, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-0.7015, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-0.8765, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-1.0518, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-1.2271, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-1.4023, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-1.5774, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-1.7524, device='cuda:0', grad_fn=<AddBackward0>)
sp_loss: -1.75235915184021

       acc     bmx  bmn      mse       spr      run
0  16.822  31.481  0.0  150.672 -3417.219  154.089
ValEpoch(000) - 
      acc    bmx  bmn     run
0  14.25  31.25  0.0  18.169
*****
 Epoch 2 of 100
TrEpoch(001) - kl tensor(-0.1898, device='cuda:0', grad_fn=<AddBackward0>)
kl tensor(-0.3770, device='cuda:0', grad_fn


       acc     bmx  bmn     mse       spr     run
0  12.065  55.942  0.0  44.901 -3479.956  48.381
ValEpoch(021) - 
      acc    bmx  bmn    run
0  12.12  53.35  0.0  7.511
*****
 Epoch 23 of 100
TrEpoch(022) - 
       acc     bmx  bmn     mse       spr     run
0  12.025  55.788  0.0  44.063 -3479.913  47.543
ValEpoch(022) - 
      acc     bmx  bmn    run
0  11.86  53.813  0.0  7.371
*****
 Epoch 24 of 100
TrEpoch(023) - 
      acc     bmx  bmn     mse       spr     run
0  11.98  55.254  0.0  43.526 -3479.599  47.006
ValEpoch(023) - 
     acc     bmx  bmn    run
0  10.6  53.755  0.0  7.279
*****
 Epoch 25 of 100
TrEpoch(024) - 
       acc     bmx  bmn     mse       spr    run
0  11.995  54.909  0.0  43.241 -3479.503  46.72
ValEpoch(024) - 
      acc     bmx  bmn    run
0  11.94  53.559  0.0  7.246
*****
 Epoch 26 of 100
TrEpoch(025) - 
      acc     bmx  bmn     mse       spr     run
0  12.01  54.919  0.0  42.963 -3479.228  46.443
ValEpoch(025) - 
      acc     bmx  bmn    run
0  11.9


       acc     bmx  bmn     mse       spr     run
0  11.585  52.825  0.0  37.179 -3473.994  40.653
ValEpoch(046) - 
     acc     bmx  bmn    run
0  11.2  51.795  0.0  6.283
*****
 Epoch 48 of 100
TrEpoch(047) - 
       acc     bmx  bmn    mse       spr     run
0  11.542  52.609  0.0  37.07 -3473.657  40.543
ValEpoch(047) - 
      acc     bmx  bmn    run
0  11.28  51.458  0.0  6.243
*****
 Epoch 49 of 100
TrEpoch(048) - 
       acc     bmx  bmn    mse       spr     run
0  11.483  52.818  0.0  36.87 -3473.515  40.343
ValEpoch(048) - 
      acc     bmx  bmn    run
0  11.17  51.005  0.0  6.208
*****
 Epoch 50 of 100
TrEpoch(049) - 
       acc     bmx  bmn     mse      spr     run
0  11.403  52.682  0.0  36.683 -3473.26  40.157
ValEpoch(049) - 
      acc     bmx  bmn    run
0  11.38  51.038  0.0  6.202
*****
 Epoch 51 of 100
TrEpoch(050) - 
       acc     bmx  bmn     mse       spr     run
0  11.373  52.632  0.0  36.577 -3473.122  40.051
ValEpoch(050) - 
      acc     bmx  bmn    run
0  11


       acc     bmx  bmn     mse       spr     run
0  11.377  47.464  0.0  33.963 -3469.121  37.432
ValEpoch(071) - 
      acc     bmx  bmn    run
0  11.17  46.165  0.0  5.776
*****
 Epoch 73 of 100
TrEpoch(072) - 
       acc    bmx  bmn    mse      spr     run
0  11.365  47.22  0.0  33.88 -3468.95  37.349
ValEpoch(072) - 
      acc     bmx  bmn    run
0  11.19  46.026  0.0  5.757
*****
 Epoch 74 of 100
TrEpoch(073) - 
       acc     bmx  bmn    mse       spr     run
0  11.375  47.463  0.0  33.82 -3468.836  37.289
ValEpoch(073) - 
      acc     bmx  bmn    run
0  11.21  45.665  0.0  5.734
*****
 Epoch 75 of 100
TrEpoch(074) - 
       acc     bmx  bmn     mse       spr     run
0  11.402  47.287  0.0  33.742 -3468.625  37.211
ValEpoch(074) - 
      acc     bmx  bmn    run
0  11.41  45.455  0.0  5.724
*****
 Epoch 76 of 100
TrEpoch(075) - 
      acc     bmx  bmn     mse       spr     run
0  11.39  47.395  0.0  33.672 -3468.585  37.141
ValEpoch(075) - 
      acc     bmx  bmn    run
0  11.2


      acc    bmx  bmn     mse       spr     run
0  11.32  45.25  0.0  32.363 -3467.771  35.831
ValEpoch(096) - 
      acc     bmx  bmn   run
0  11.13  43.387  0.0  5.53
*****
 Epoch 98 of 100
TrEpoch(097) - 
       acc     bmx  bmn     mse       spr     run
0  11.255  45.115  0.0  32.309 -3467.714  35.776
ValEpoch(097) - 
      acc     bmx  bmn    run
0  11.12  43.446  0.0  5.511
*****
 Epoch 99 of 100
TrEpoch(098) - 
       acc     bmx  bmn     mse      spr     run
0  11.262  44.998  0.0  32.272 -3467.47  35.739
ValEpoch(098) - 
      acc     bmx  bmn    run
0  10.98  43.022  0.0  5.495
*****
 Epoch 100 of 100
TrEpoch(099) - 
       acc     bmx  bmn     mse       spr     run
0  11.232  45.242  0.0  32.227 -3467.317  35.694
ValEpoch(099) - 
      acc     bmx  bmn    run
0  10.98  43.017  0.0  5.491
33.9 minutes


In [14]:
print(result_df_tr_all)

       acc     bmx  bmn      mse       spr      run
0   16.822  31.481  0.0  150.672 -3417.219  154.089
1   18.418  37.704  0.0  101.894 -3451.070  105.345
2   13.130  41.888  0.0   83.850 -3464.002   87.314
3   17.645  42.489  0.0   75.844 -3467.321   79.312
4   17.640  44.134  0.0   71.300 -3468.729   74.769
5   12.952  47.107  0.0   67.448 -3470.906   70.919
6   16.697  48.202  0.0   64.615 -3473.077   68.088
7   15.670  49.320  0.0   63.005 -3474.386   66.480
8   15.027  49.679  0.0   60.060 -3474.755   63.535
9   13.820  51.390  0.0   57.637 -3475.614   61.113
10  12.750  52.259  0.0   56.285 -3476.353   59.761
11  12.207  52.748  0.0   55.542 -3476.645   59.019
12  11.868  53.516  0.0   54.667 -3476.632   58.144
13  11.667  53.646  0.0   53.916 -3476.634   57.393
14  11.463  54.048  0.0   51.954 -3476.897   55.431
15  11.057  54.681  0.0   49.717 -3477.869   53.195
16  12.310  55.212  0.0   48.982 -3478.584   52.461
17  12.393  54.975  0.0   47.888 -3478.888   51.367
18  12.337  

In [15]:
print(result_df_va_all)

      acc     bmx  bmn     run
0   14.25  31.250  0.0  18.169
1   17.52  35.992  0.0  15.530
2   17.37  41.532  0.0  13.218
3   17.70  42.670  0.0  12.123
4   17.43  44.112  0.0  11.731
5   17.13  46.047  0.0  10.979
6   15.84  47.428  0.0  10.660
7   15.81  47.965  0.0  10.420
8   14.35  49.240  0.0   9.859
9   13.62  50.880  0.0   9.495
10  12.14  51.555  0.0   9.358
11  11.87  51.652  0.0   9.259
12  11.58  51.823  0.0   9.070
13  11.55  51.779  0.0   8.986
14  11.45  52.281  0.0   8.445
15  12.06  53.204  0.0   8.258
16  12.15  53.204  0.0   8.177
17  12.43  53.521  0.0   7.982
18  12.30  53.318  0.0   7.849
19  12.16  53.100  0.0   7.779
20  11.94  53.380  0.0   7.564
21  12.12  53.350  0.0   7.511
22  11.86  53.813  0.0   7.371
23  10.60  53.755  0.0   7.279
24  11.94  53.559  0.0   7.246
25  11.98  53.568  0.0   7.223
26  11.90  53.438  0.0   6.998
27  10.59  53.396  0.0   6.973
28  12.07  53.161  0.0   6.944
29  12.06  52.438  0.0   6.799
30  11.93  52.292  0.0   6.765
31  12.0